In [19]:
from IPython.display import display
import pandas as pd
data = pd.read_csv('data/task1/output/source_csvs/finnish-train-low')
del data['Unnamed: 0']
# data = data.dropna()
source_words = data[data['pos'] == 'N']['source'].unique()
len(source_words)

48

In [20]:
alphabets = set()
for source in source_words:
    for alphabet in source:
        alphabets.add(alphabet)
print(alphabets, len(alphabets))
len(source_words)

{'d', 'o', 'l', 'ä', 'š', 'y', 'b', 'u', 'k', 'a', 's', 'm', 't', 'h', 'ö', 'r', 'v', 'e', 'p', 'g', 'c', 'j', 'f', 'n', 'i'} 25


48

In [21]:
mapping = dict()
for source in source_words:
    l = len(source)
    if l not in mapping.keys():
        mapping[l] = 1
        continue
    mapping[l] += 1
mapping

{4: 1,
 5: 7,
 6: 2,
 7: 7,
 8: 4,
 9: 4,
 10: 5,
 11: 3,
 12: 5,
 13: 2,
 14: 3,
 16: 3,
 18: 1,
 19: 1}

In [33]:
EPSILON = 5  # Looking at the above frequency distribution
Ci = len(alphabets)
cumulative_count = 0
for i, word_len in enumerate(sorted(mapping.keys())):
    cumulative_count += mapping[word_len]
    print(word_len, cumulative_count)
    if cumulative_count > 0.7 * len(source_words):
        EPSILON = word_len
        break

4 1
5 8
6 10
7 17
8 21
9 25
10 30
11 33
12 38


In [70]:
import numpy as np
# *****************Start-Generate Matrix*******************

# Function to generate character id of the form char_i_j_k
def gen_cid(char, L='', R=''):
    """
    Returns charater id:
    char -> character
    L -> Postion of character from LEFT
    R -> Postion of character from RIGHT(negative)
    """
    return(char + '_' + str(L) + '_' + str(R))

# Generate character ids
cids = list()
for c in alphabets:
    for i in range(EPSILON + 1):
        for j in range(EPSILON + 1):
            cid = gen_cid(c, i, -j)
            cids.append(cid)
#     cids.append(gen_cid(c))
#     for o in range(1, EPSILON+1):
#         cids.append(gen_cid(c, '', -o))
#     for j in range(EPSILON):
#         cids.append(gen_cid(c, j, ''))
assert len(cids) == Ci*pow(EPSILON + 1, 2)  # Check for the correct number of entries



In [32]:
# Initialize Adjaceny Matrix
adj_mat = pd.DataFrame(data=np.zeros(shape=(len(cids), len(cids))), columns=cids, index=cids)

def cid_exists(cid, matrix):
    """Checks if cid is present in matrix's keys"""
    return(True if cid in matrix else False)

# ToDo: Multithreading
for source in source_words:
    s_l = len(source)
    for i in range(0, s_l):
        c_1 = source[i]
#         cid_1 = gen_cid(c_1, i + 1, i - s_l)

        # cid_1 subset
        subset_cid_1 = [gen_cid(c_1, i + 1, i - s_l), gen_cid(c_1, 0, i - s_l), gen_cid(c_1, i, 0), gen_cid(c_1, 0, 0)]

        # for (j, c_2) in enumerate(source):        
        for j in range(i+1, s_l):
            c_2 = source[j]

#             cid_2 = gen_cid(c_2, j, j - s_l)
            
            cid_pairs = []

            # Case 1 (1): cid_1 --> cid_2
#             cid_pairs.append((cid_1, cid_2))

            # cid_2 subset_
            subset_cid_2 = [gen_cid(c_2, j + 1, j - s_l), gen_cid(c_2, 0, j - s_l), gen_cid(c_2, j + 1, 0), gen_cid(c_2, 0, 0)]
            
            # Case 2 (3): cid_1 --> sub(cid_2)
#             for cid in subset_cid_2:
#                 cid_pairs.append((cid_1, cid))
            
            # Case 3 (3): sub(cid_1) --> cid_2
#             for cid in subset_cid_1:
#                 cid_pairs.append((cid, cid_2))
            
            # Case 4 (9): sub(cid_1) --> sub(cid_2)
            for s_cid_1 in subset_cid_1:
                for s_cid_2 in subset_cid_2:
                    try:
                        adj_mat[s_cid_1][s_cid_2] += 1
                        adj_mat[s_cid_2][s_cid_1] += 1
                    except KeyError:
                        pass
                        
#                     cid_pairs.append((s_cid_1, s_cid_2))
            
#             for (cid1, cid2) in cid_pairs:
#                 try:
#                     adj_mat[cid1][cid2] += 1
#                     adj_mat[cid2][cid1] += 1
#                 except KeyError:
#                     pass
#             for cid2 in subset_cid_2:
#                 try:
#                     adj_mat[c_1 + '__'][cid2] += 1
#                     adj_mat[cid2][c_1 + '__'] += 1
#                 except KeyError:
#                     pass
#             for cid1 in subset_cid_1:
#                 try:
#                     adj_mat[cid1][c_2 + '__'] += 1
#                     adj_mat[c_2 + '__'][cid1] += 1
#                 except KeyError:
#                     pass
#             adj_mat[c_1 + '__'][c_2 + '__'] += 1
#             adj_mat[c_2 + '__'][c_1 + '__'] += 1

In [34]:
tot = adj_mat.max().sum()
adj_mat.max().apply(lambda x: x/tot).sort_values(ascending=False)
adj_mat.max().sort_values(ascending=False)
adj_mat

,و_0_0,و_0_-1,و_0_-2,و_0_-3,و_0_-4,و_0_-5,و_0_-6,و_0_-7,و_0_-8,و_0_-9,...,س_9_0,س_9_-1,س_9_-2,س_9_-3,س_9_-4,س_9_-5,س_9_-6,س_9_-7,س_9_-8,س_9_-9
و_0_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
و_0_-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
و_0_-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
و_0_-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
و_0_-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
و_0_-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
و_0_-6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
و_0_-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
و_0_-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
و_0_-9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
pd.read_csv

<function pandas.io.parsers._make_parser_function.<locals>.parser_f>